In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

URL = "https://wikiroutes.info/es/lima/catalog"


def normalizar(texto: str) -> str:
    """Limpia el texto de cada ficha de ruta."""
    return " ".join(texto.split())


def esperar_chips(driver, timeout=60):
    """Espera a que haya al menos un chip de ruta en la página."""
    wait = WebDriverWait(driver, timeout)
    wait.until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "a.city.tag-btn.tag-btn--float")
        )
    )


def expandir_listas(driver, timeout=60):
    """
    Hace clic en todos los botones visibles de 'Expandir la lista'
    hasta que ya no quede ninguno visible.
    """
    wait = WebDriverWait(driver, timeout)

    try:
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.button-more.expandFullList")
            )
        )
    except TimeoutException:
        return

    while True:
        botones = [
            b
            for b in driver.find_elements(
                By.CSS_SELECTOR, "div.button-more.expandFullList"
            )
            if b.is_displayed()
        ]
        if not botones:
            break

        for b in botones:
            try:
                driver.execute_script("arguments[0].click();", b)
            except Exception:
                pass

        time.sleep(1)


def obtener_rutas_visibles(driver):
    """Devuelve lista de rutas visibles (texto de cada chip)."""
    chips = driver.find_elements(By.CSS_SELECTOR, "a.city.tag-btn.tag-btn--float")
    rutas = []
    vistos = set()

    for chip in chips:
        if not chip.is_displayed():
            continue
        txt = normalizar(chip.text)
        if txt and txt not in vistos:
            vistos.add(txt)
            rutas.append(txt)

    return rutas


def main():
    # Navegador (sin headless)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 60)

    try:
        driver.get(URL)
        time.sleep(3)

        # Asegurar pestaña "Todas las rutas"
        try:
            tab_todas = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "span#tabs-btn-1"))
            )
            driver.execute_script("arguments[0].click();", tab_todas)
        except TimeoutException:
            pass  # normalmente ya está seleccionada

        # Esperar a que aparezcan chips
        esperar_chips(driver)

        # Expandir todas las listas con el estado inicial (solo activas)
        expandir_listas(driver)

        # Rutas activas
        rutas_activas = obtener_rutas_visibles(driver)
        print(f"Total rutas activas: {len(rutas_activas)}")
        for r in rutas_activas:
            print(r)

        # Click en el label "Mostrar rutas inactivas"
        label_inactivas = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "label[for='checkboxShowInactive']")
            )
        )
        driver.execute_script("arguments[0].click();", label_inactivas)
        time.sleep(2)

        # Expandir otra vez (pueden aparecer más bloques)
        expandir_listas(driver)

        # Rutas totales (activas + inactivas)
        rutas_totales = obtener_rutas_visibles(driver)
        print(f"\nTotal rutas totales (activas + inactivas): {len(rutas_totales)}")

        # Calcular inactivas: totales - activas
        set_activas = set(rutas_activas)
        rutas_inactivas = [r for r in rutas_totales if r not in set_activas]

        print(f"\nTotal rutas inactivas: {len(rutas_inactivas)}")
        for r in rutas_inactivas:
            print(r)

        # Por si quieres usarlas luego en tu código:
        return rutas_activas, rutas_totales, rutas_inactivas

    finally:
        driver.quit()


if __name__ == "__main__":
    rutas_activas, rutas_totales, rutas_inactivas = main()


Total rutas activas: 418
1 (Expreso San Isidro)
2 (Expreso San Isidro)
018p
204
206
209
301
303
305
336
370
371
372
404
405
406
412
1003
1005
1006
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1023
1024
1025
1027
1028
1029
1031
1033
1034
1035
1036
1037
1038
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1087
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1137
1138
1145
1160
1166
1168
1176
1177
1178
1179
1180
1182
1183
1185
1187
1191
1192
1193
1194
1195
1199
1200
1204
1205
1219
1220
1221
1222
1224
1229
1231
1233
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1246
1247
1248
1249
1250
1251
1252
1253
1255
1285 (Lima - Callao)
1288
1289
1320
1322
1323
1324
1325
1326
13

In [ ]:
print(rutas_activas)

['1 (Expreso San Isidro)', '2 (Expreso San Isidro)', '018p', '204', '206', '209', '301', '303', '305', '336', '370', '371', '372', '404', '405', '406', '412', '1003', '1005', '1006', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1023', '1024', '1025', '1027', '1028', '1029', '1031', '1033', '1034', '1035', '1036', '1037', '1038', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '1070', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '1078', '1079', '1080', '1081', '1082', '1083', '1084', '1087', '1089', '1090', '1091', '1092', '1093', '1094', '1095', '1096', '1097', '1098', '1099', '1100', '1107', '1108', '1109', '1110', '1111', '1112', '1113', '1114', '1115', '1116', '1117', '1118', '1119', '1120', '1121', '1122', '1123', '1124', '

: 